In [ ]:
# Dependencies
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from datetime import datetime, UTC
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    # now = datetime.now(UTC).isoformat() + "Z"  # 'Z' indicates UTC time
    # now = datetime.utcnow().isoformat() + "Z"
    now = datetime.now(UTC).isoformat().rsplit("+", 1)[0] + "Z"
    print("Getting the upcoming 10 events \n now: ", now)
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

---

In [2]:

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


def get_credentials():
  creds = None
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

    return creds

In [5]:
from googleapiclient.discovery import build


creds = get_credentials()
service = build("calendar", "v3", credentials=creds)    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57724%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=sDrsBJgqErPXoCWt0OuIJfjKsNbq6Z&access_type=offline


In [20]:
def get_calendars_info():
  # Call the Calendar API
  calendars = service.calendarList().list().execute()
  data = []
  for item in calendars.get('items', []):
    if item['accessRole'] != 'reader':
      data.append({'kind': item['kind'], 'id': item['id'], 'timeZone': item['timeZone']})
  return data


get_calendars_info()

[{'kind': 'calendar#calendarListEntry',
  'id': 'jorgeang33@gmail.com',
  'timeZone': 'America/Mexico_City'},
 {'kind': 'calendar#calendarListEntry',
  'id': 'stdevelopersyelling@gmail.com',
  'timeZone': 'America/Mexico_City'},
 {'kind': 'calendar#calendarListEntry',
  'id': '9a1c2703ead6fd80455f9e1eb04158669a60d7ab77a8a29ea9ae8d530954b4f7@group.calendar.google.com',
  'timeZone': 'America/Mexico_City'}]

In [23]:
from typing import Optional, Union, List, Dict, TypeAlias, Literal

def create_event_body(summary: str, 
                      start_time: str,
                      end_time: str,
                      timezone: str,
                      location: Optional[str] = None,
                      description: Optional[str] = None,
                      recurrence: Optional[List[str]] = None,
                      attendees: Optional[List[Dict[str, str]]] = None,
                      reminders: Optional[Dict[str, Union[bool, List[Dict[str, Union[str, int]]]]]] = None,
                      conference_data: Optional[Dict[str, dict]] = None 
                      ):
    return {
        "summary": summary,
        "location": location,
        "description": description,
        "start": {"dateTime": start_time, "timeZone": timezone},
        "end": {"dateTime": end_time, "timeZone": timezone},
        "recurrence": recurrence,
        "attendees": attendees,
        "reminders": reminders,
        "conferenceData": conference_data
    }

In [24]:
from typing import Dict, TypeAlias, Literal


ConferenceData: TypeAlias = Literal[0, 1]
# 0: No permite crear una reunión con Google Meet
# 1: Permite crear una reunión con Google Meet

def create_event(body: Dict, 
                 calendarId: str = 'primary', 
                 conferenceData: ConferenceData = 1) -> str:
    
    event = service.events().insert(calendarId=calendarId, 
                                    body=body, 
                                    conferenceDataVersion=conferenceData).execute()
    return event.get('htmlLink')

In [33]:
from datetime import datetime, timedelta

def get_current_time():
    return datetime.now().astimezone().replace(microsecond=0).isoformat()

def plus_time(current_datetime: str, minutes: int):
    date_object = datetime.fromisoformat(current_datetime).strftime('%Y-%m-%d %H:%M:%S')
    new_date = datetime.strptime(date_object, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=minutes)
    return new_date.replace(microsecond=0).isoformat()

In [34]:
now = get_current_time()
end = plus_time(now, 30)

event_body = create_event_body(summary='Test Event', start_time=now, end_time=end, timezone='America/Mexico_City')

link = create_event(event_body)

print(link)

https://www.google.com/calendar/event?eid=OGZ0NHBtOGllbjIzbW1yZHUxM3RsM2JkNGMgam9yZ2VhbmczM0Bt


In [22]:
# Refer to the Python quickstart on how to setup the environment:
# https://developers.google.com/calendar/quickstart/python
# Change the scope to 'https://www.googleapis.com/auth/calendar' and delete any
# stored credentials.

import uuid

# event = {
#   'summary': 'Evento Random',
#   'location': '800 Howard St., San Francisco, CA 94103',
#   'description': 'A chance to hear more about Google\'s developer products.',
#   'start': {
#     'dateTime': '2024-10-18T19:52:11-07:00',
#     'timeZone': 'America/Los_Angeles',
#   },
#   'end': {
#     'dateTime': '2024-10-18T20:52:11-07:00',
#     'timeZone': 'America/Los_Angeles',
#   },
#   'recurrence': [
#     'RRULE:FREQ=DAILY;COUNT=2'
#   ],
#   'attendees': [
#     {'email': 'lpage@example.com'},
#     {'email': 'sbrin@example.com'},
#   ],
#   'reminders': {
#     'useDefault': False,
#     'overrides': [
#       {'method': 'email', 'minutes': 24 * 60},
#       {'method': 'popup', 'minutes': 10},
#     ],
#   },
#   "conferenceData": {
#       "createRequest": {
#         "requestId": str(uuid.uuid4()), # Necesario para que funcione
#         "conferenceSolutionKey": {
#           "type": "hangoutsMeet"
#         }
#       },
#   }
# }

event = {
  'summary': 'Evento Random',
  'location': None, 
  'description': None, 
  'start': {
    'dateTime': '2024-10-18T21:52:11-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2024-10-18T22:52:11-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': None,
  'attendees': None,
  'reminders': None,
  "conferenceData": None
}

event = service.events().insert(calendarId='primary', body=event, conferenceDataVersion=1).execute()
service.events
print ('Event created: %s' % (event.get('htmlLink')))



Event created: https://www.google.com/calendar/event?eid=cmd0Y2NldnU0cHRiYjl1a2pyNTltdWRkaTggam9yZ2VhbmczM0Bt
